In [1]:

import xlrd
import pymysql
import math
import pandas as pd
from sqlalchemy import create_engine
import time

from bs4 import BeautifulSoup 
import requests 
import re 
import random
import pandas as pd 
import pymysql  
from sqlalchemy import create_engine





In [2]:
conn = pymysql.connect(
        host='localhost', 
        user='root', 
        passwd = 'fc117108',
        db='music',
        port=3306,
        charset='utf8'
        )
cur = conn.cursor()
cur.execute("SET NAMES utf8")

sql_cmd = "SELECT * from music.new_singer_name;"
df = pd.read_sql(sql_cmd,conn)
df



/usr/local/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,id,singer,title,url,top
0,429,no,戒煙不如戒命,https://mojim.com/twy104622x3x1.htm,心傑
1,430,no,彩虹 2011,https://mojim.com/twy104622x2x1.htm,張心傑
2,431,no,愛了就沉淪,https://mojim.com/twy104622x2x2.htm,張心傑
3,432,no,人形立牌,https://mojim.com/twy104622x2x3.htm,張心傑
4,433,no,最愛不是我,https://mojim.com/twy104622x2x4.htm,張心傑
...,...,...,...,...,...
28519,293032,no,敲鑼打鼓,https://mojim.com/twy106299x35x28.htm,合輯(2009年發行)
28520,293033,主唱：容祖兒．倉木麻衣．林俊傑．張敬軒．泳兒．陳偉霆．徐子珊．鄧紫棋,Cheer up!,https://mojim.com/twy106299x17x1.htm,合輯(2009年發行)
28521,293034,no,過去的明天,https://mojim.com/twy106299x17x2.htm,合輯(2009年發行)
28522,293035,主唱：吳克群．光良．許茹芸．溫嵐．周蕙．吳建豪,暮光操場,https://mojim.com/twy106299x17x3.htm,合輯(2009年發行)


In [24]:
singer_id=[]
for i in df[:].url:
    values=i.split('/twy')[1].split('x')[0]
    singer_id.append(values)
df['singer_id']=singer_id
df

,id,singer,title,url,top,singer_id
0,429,no,戒煙不如戒命,https://mojim.com/twy104622x3x1.htm,心傑,104622
1,430,no,彩虹 2011,https://mojim.com/twy104622x2x1.htm,張心傑,104622
2,431,no,愛了就沉淪,https://mojim.com/twy104622x2x2.htm,張心傑,104622
3,432,no,人形立牌,https://mojim.com/twy104622x2x3.htm,張心傑,104622
4,433,no,最愛不是我,https://mojim.com/twy104622x2x4.htm,張心傑,104622
...,...,...,...,...,...,...
28519,293032,no,敲鑼打鼓,https://mojim.com/twy106299x35x28.htm,合輯(2009年發行),106299
28520,293033,主唱：容祖兒．倉木麻衣．林俊傑．張敬軒．泳兒．陳偉霆．徐子珊．鄧紫棋,Cheer up!,https://mojim.com/twy106299x17x1.htm,合輯(2009年發行),106299
28521,293034,no,過去的明天,https://mojim.com/twy106299x17x2.htm,合輯(2009年發行),106299
28522,293035,主唱：吳克群．光良．許茹芸．溫嵐．周蕙．吳建豪,暮光操場,https://mojim.com/twy106299x17x3.htm,合輯(2009年發行),106299


In [27]:
df1=df[['singer_id','top']].drop_duplicates()
df1

,singer_id,top
0,104622,心傑
1,104622,張心傑
18,158007,聶傑銘
19,158007,聶杰銘
21,134114,阿山
...,...,...
27870,106299,合輯(2009年發行)
27971,106299,潮間帶音樂盒
28008,106299,1.COM(冠軍帝國)
28270,106299,2009太陽計劃主題曲


In [29]:
data_dict=df1.to_dict('records')
print(len(data_dict))
print(data_dict[0])
indices=dict()

for doc in data_dict[:]:
#     print(doc)
   
        if doc['singer_id'] not in indices:
            indices[doc['singer_id']]=[]
        indices[doc['singer_id']].append(str(doc['top']))
# indices



list_key_values=[[k,v] for k ,v in indices.items()]

df_if=pd.DataFrame.from_dict(list_key_values)
df_if.columns=['singer_id','singer_name']


df_if['count']=[len(l) for l in df_if.singer_name] 
df_if=df_if.sort_values('count')
df_if[df_if['count']>1]


1064
{'singer_id': '104622', 'top': '心傑'}


,singer_id,singer_name,count
177,110113,"[嘴哥樂團, MISTERMOUTH]",2
178,117675,"[太妃堂TOFFEE, 太妃堂]",2
172,118874,"[馬克白, Macbeth]",2
175,113322,"[陳冠希+MC仁+廚房仔, 陳冠希xMC仁x廚房仔]",2
171,176416,"[Kawa, KAWA樂隊]",2
...,...,...,...
242,124806,"[櫻桃紅之袖珍媽媽, 黑暗中的笑臉, 詐欺遊戲, 22K夢想高飛, 驕陽似我, 青是受傷春是...",49
210,180933,"[漢創群星(HCSTARS), MOMO音樂計畫, 香港人(HongKongPeople),...",52
275,161281,"[守護麗人, TheK2, 青雲志2, 獨家保鑣, 浮士德的微笑, 孤芳不自賞, 惡作劇之吻...",57
268,181695,"[MadDog, 獨步天下, 端腦, 瑪麗蘇遇上傑克蘇, BpRania, 可惜不是你, 戲...",63


In [2]:
conn = pymysql.connect(
        host='localhost', 
        user='root', 
        passwd = 'fc117108',
        db='music',
        port=3306,
        charset='utf8'
        )
cur = conn.cursor()
cur.execute("SET NAMES utf8")

sql_cmd = "SELECT distinct same_singer_id,singer_name FROM music.Excel10;"
df = pd.read_sql(sql_cmd,conn)
# df

sql_cmd = "SELECT distinct link,same_singer_id,singer_name,id  FROM music.Excel10;"
df1 = pd.read_sql(sql_cmd,conn)
# df1



/usr/local/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


In [3]:
data_dict=df.to_dict('records')
print(len(data_dict))
print(data_dict[0])
indices=dict()

for doc in data_dict[:]:
#     print(doc)
   
        if doc['same_singer_id'] not in indices:
            indices[doc['same_singer_id']]=[]
        indices[doc['same_singer_id']].append(str(doc['singer_name']))
# indices



list_key_values=[[k,v] for k ,v in indices.items()]

df_if=pd.DataFrame.from_dict(list_key_values)
df_if.columns=['same_singer_id','singer_name']
df_if

df_if['count']=[len(l) for l in df_if.singer_name] 
df_if



9099
{'same_singer_id': 100198, 'singer_name': '張洪量'}


,same_singer_id,singer_name,count
0,100198,[張洪量],1
1,100146,[葉璦菱],1
2,100431,[費翔],1
3,100362,[楊采妮],1
4,122502,[陳浩德],1
...,...,...,...
8235,160726,[Kinn],1
8236,169267,[桃河りか],1
8237,186221,[丸山和歌子],1
8238,134729,[ChiliTomato],1


In [77]:

df_if=df_if.sort_values('count')
same=df_if[df_if['count']>1].same_singer_id.tolist()
print(len(same))




282


In [107]:
df2=df1[df1.same_singer_id.isin(same)]
print('df1',len(df1))
print('df2',len(df2))
df3=df2[['id','link']]
df4=df3[14266:]
print('df3',len(df3))
same2=df3.set_index('id').T.to_dict('list')
same1=df4.set_index('id').T.to_dict('list')
print(len(same2),len(same1))

df1 261489
df2 28524
df3 28524
28524 14258


In [102]:
index1=list(same2.keys()).index(280781)
print('index1',index1)
for i ,(k,v) in enumerate(same1.items()):
             
#     print(k,v)


index1 14266
280781 ['https://mojim.com/twy133743x15x16.htm']
280783 ['https://mojim.com/twy133743x15x18.htm']
280766 ['https://mojim.com/twy133743x15x1.htm']
280768 ['https://mojim.com/twy133743x15x3.htm']


In [108]:
result=[]
start = time.time()
conn = pymysql.connect(
        host='localhost', 
        user='root', 
        passwd = 'fc117108',
        db='music',
        port=3306,
        charset='utf8'
        )
cur = conn.cursor()



for key,val in same1.items():
    singer='no'
    title='no'
    top='no'
    try:
        url = val[0]
        resp = requests.get(url)
        
        soup = BeautifulSoup(resp.text, 'html.parser')
        txt=soup.find(id='fsZx3')
        txt2=soup.find_all('a',href=True)
        txt3=soup.find('dl',id='fsZx1')
        url_cut=url.split('com')[1]
#         print('txt',txt)
        if txt==None or txt2==None or txt3==None:
            pass
        else:
        
            for i,val in enumerate(txt3) :
                if i==0:
                    top=re.sub(r"\s+", "", val.string)
            for i in txt:
    #             print('i',i)
                if '演唱' in i:
                    singer=i
                elif '主唱' in i :
                    singer=i


            for i in txt2:
                if i['href']==url:
                    title=i.string
                elif i['href']==url_cut:
                    title=i.string

        values=[key,singer,title,url,top]
#         print('values',values)
        sql = "INSERT IGNORE INTO new_singer_name (id,singer, title,url,top) VALUES (%s, %s, %s, %s, %s)"

        conn = pymysql.connect(
            host='localhost', 
            user='root', 
            passwd = 'fc117108',
            db='music',
            port=3306,
            charset='utf8'
            )
        cur = conn.cursor()


        cur.execute(sql, values)

        conn.commit()
    except:
        print('error',key,val.string,url,url_cut)
print('end')

end


In [18]:
url = 'https://mojim.com/twy104727x17x35.htm'
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')
txt=soup.find(id='fsZx3')
txt2=soup.find_all('a',href=True)
txt3=soup.find('dl',id='fsZx1')
url_cut=url.split('com')[1]
for i,val in enumerate(txt3) :
    if i==0:
        top=re.sub(r"\s+", "", val.string)
for i in txt:
    if '演唱' in i:
        singer=i
    elif '主唱' in i :
        singer=i


for i in txt2:
    if i['href']==url:
        title=i.string
    elif i['href']==url_cut:
        title=i.string

values=[key,singer,title,url,top]
print('values',values)

In [ ]:

conn = pymysql.connect(
        host='localhost', 
        user='root', 
        passwd = 'fc117108',
        db='music',
        port=3306,
        charset='utf8'
        )
cur = conn.cursor()
cur.execute("SET NAMES utf8")

sql_cmd = "SELECT * FROM music.Excel10;"
df = pd.read_sql(sql_cmd,conn)
# df

sql_cmd = "SELECT *   FROM music.new_singer_name;"
df1 = pd.read_sql(sql_cmd,conn)
# df1


df1.loc[df1['singer']=='no','singer']=df1.title


